In [63]:
import wandb
wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [64]:
import os
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check mmcv installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

1.7.0 True
11.0
GCC 7.3


In [65]:
# 모듈 import
import mmdet
from mmdet.apis import set_random_seed
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset, replace_ImageToTensor)

In [66]:
seed = 42

set_random_seed(seed, deterministic=False)
os.environ['PYTHONHASHSEED'] = str(seed)

In [67]:
from mmcv import Config

# config file 들고오기
baseline_cfg_path = './configs/faster_rcnn/faster_rcnn_r50_fpn_1x_coco.py'
cfg = Config.fromfile(baseline_cfg_path)

In [68]:
model_name = 'faster_rcnn_r50_fpn_1x_coco'
exp = 'originalsize'

# Folder to store model logs and weight files
job_folder = f'./working/exp[{exp}]_{model_name}'
cfg.work_dir = job_folder

# Change the wnd username and project name below
wnb_username = 'sunhyuk'
wnb_project_name = 'cv-3-bitcoin'

if not os.path.exists(job_folder):
    os.makedirs(job_folder)

print("Job folder:", job_folder)

Job folder: ./working/exp[originalsize]_faster_rcnn_r50_fpn_1x_coco


In [69]:
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")
root='../../dataset/'

# dataset config 수정
cfg.data.train.classes = classes
cfg.data.train.img_prefix = root
cfg.data.train.ann_file = root + 'cv_train_1.json' # train json 정보
cfg.data.train.pipeline[2]['img_scale'] = (1024,1024) # Resize

cfg.data.val.classes = classes
cfg.data.val.img_prefix = root
cfg.data.val.ann_file = root + 'cv_val_1.json' # test json 정보
cfg.data.val.pipeline[1]['img_scale'] = (1024,1024) # Resize

cfg.data.test.classes = classes
cfg.data.test.img_prefix = root
cfg.data.test.ann_file = root + 'test.json' # test json 정보
cfg.data.test.pipeline[1]['img_scale'] = (1024,1024) # Resize

cfg.data.samples_per_gpu = 8
cfg.data.workers_per_gpu = 4

cfg.seed = seed #~~~
cfg.gpu_ids = [0]
#cfg.work_dir = './work_dirs/faster_rcnn_r50_fpn_1x_trash'

cfg.model.roi_head.bbox_head.num_classes = 10 ##클래수 수

cfg.evaluation = dict(
    interval=1,
    save_best='auto',
    metric='bbox',
    classwise=True,
)
    
# cfg.optimizer = dict(type='SGD', lr=0.01, momentum=0.9, weight_decay=0.0001)
cfg.optimizer = dict(type='Adam', lr=8e-5)
cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.checkpoint_config = dict(max_keep_ckpts=3, interval=1)
cfg.lr_config = dict(
    policy='step',
    warmup='linear',
    warmup_iters=50,
    warmup_ratio=0.001,
    step=[8, 11]
)

cfg.runner.max_epochs = 12

cfg.log_config.hooks = [
    dict(type='TextLoggerHook'),
    dict(type='WandbLoggerHook',
         interval=10,
         init_kwargs=dict(project=wnb_username,entity=wnb_project_name, name=f'[{exp}]_{model_name}')
        )
]
cfg.log_config.interval = 10
# cfg.checkpoint_config = 20
# cfg.fp16 = dict(loss_scale=512.)
print(cfg.pretty_text)


model = dict(
    type='FasterRCNN',
    backbone=dict(
        type='ResNet',
        depth=50,
        num_stages=4,
        out_indices=(0, 1, 2, 3),
        frozen_stages=1,
        norm_cfg=dict(type='BN', requires_grad=True),
        norm_eval=True,
        style='pytorch',
        init_cfg=dict(type='Pretrained', checkpoint='torchvision://resnet50')),
    neck=dict(
        type='FPN',
        in_channels=[256, 512, 1024, 2048],
        out_channels=256,
        num_outs=5),
    rpn_head=dict(
        type='RPNHead',
        in_channels=256,
        feat_channels=256,
        anchor_generator=dict(
            type='AnchorGenerator',
            scales=[8],
            ratios=[0.5, 1.0, 2.0],
            strides=[4, 8, 16, 32, 64]),
        bbox_coder=dict(
            type='DeltaXYWHBBoxCoder',
            target_means=[0.0, 0.0, 0.0, 0.0],
            target_stds=[1.0, 1.0, 1.0, 1.0]),
        loss_cls=dict(
            type='CrossEntropyLoss', use_sigmoid=True, loss_weight=1.

In [70]:
# build_dataset
datasets = [build_dataset(cfg.data.train)]

# dataset 확인
datasets[0]

loading annotations into memory...
Done (t=0.05s)
creating index...
index created!



CocoDataset Train dataset with number of images 3677, and instance counts: 
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| category          | count | category      | count | category        | count | category    | count | category     | count |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| 0 [General trash] | 2993  | 1 [Paper]     | 4685  | 2 [Paper pack]  | 681   | 3 [Metal]   | 711   | 4 [Glass]    | 733   |
| 5 [Plastic]       | 2211  | 6 [Styrofoam] | 970   | 7 [Plastic bag] | 3808  | 8 [Battery] | 136   | 9 [Clothing] | 378   |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+

In [71]:
# 모델 build 및 pretrained network 불러오기
model = build_detector(cfg.model)
model.init_weights()

2022-04-03 04:27:58,681 - mmcv - INFO - initialize ResNet with init_cfg {'type': 'Pretrained', 'checkpoint': 'torchvision://resnet50'}
2022-04-03 04:27:58,683 - mmcv - INFO - load model from: torchvision://resnet50
2022-04-03 04:27:58,684 - mmcv - INFO - load checkpoint from torchvision path: torchvision://resnet50
2022-04-03 04:27:58,909 - mmcv - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias

2022-04-03 04:27:58,934 - mmcv - INFO - initialize FPN with init_cfg {'type': 'Xavier', 'layer': 'Conv2d', 'distribution': 'uniform'}
2022-04-03 04:27:58,962 - mmcv - INFO - initialize RPNHead with init_cfg {'type': 'Normal', 'layer': 'Conv2d', 'std': 0.01}
2022-04-03 04:27:58,972 - mmcv - INFO - initialize Shared2FCBBoxHead with init_cfg [{'type': 'Normal', 'std': 0.01, 'override': {'name': 'fc_cls'}}, {'type': 'Normal', 'std': 0.001, 'override': {'name': 'fc_reg'}}, {'type': 'Xavier', 'distribution': 'uniform', 'override'

In [72]:
# 모델 학습
train_detector(model, datasets[0], cfg=cfg, distributed=False, validate=True)

2022-04-03 04:27:59,418 - mmdet - INFO - Start running, host: root@642ef2a3bc92, work_dir: /opt/ml/detection/baseline/mmdetection/working/exp[originalsize]_faster_rcnn_r50_fpn_1x_coco
2022-04-03 04:27:59,419 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) WandbLoggerHook                    
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) NumClassCheckHook                  
(LOW         ) IterTimerHook                      
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) WandbLoggerHook                    
 -------------------- 
before_train_iter:
(VERY_HIGH   ) StepLrUpdaterHook                  
(LOW         ) Iter

loading annotations into memory...
Done (t=0.02s)
creating index...
index created!


2022-04-03 04:28:24,068 - mmdet - INFO - Epoch [1][10/460]	lr: 1.447e-05, eta: 2:28:20, time: 1.615, data_time: 0.293, memory: 31139, loss_rpn_cls: 0.6919, loss_rpn_bbox: 0.0531, loss_cls: 1.9473, acc: 44.1504, loss_bbox: 0.0181, loss: 2.7105, grad_norm: 22.2592
2022-04-03 04:28:37,311 - mmdet - INFO - Epoch [1][20/460]	lr: 3.045e-05, eta: 2:14:43, time: 1.324, data_time: 0.020, memory: 31139, loss_rpn_cls: 0.4859, loss_rpn_bbox: 0.0785, loss_cls: 0.5168, acc: 96.2549, loss_bbox: 0.0893, loss: 1.1705, grad_norm: 11.1888
2022-04-03 04:28:50,981 - mmdet - INFO - Epoch [1][30/460]	lr: 4.643e-05, eta: 2:11:21, time: 1.367, data_time: 0.020, memory: 31139, loss_rpn_cls: 0.2204, loss_rpn_bbox: 0.0795, loss_cls: 0.4410, acc: 95.0854, loss_bbox: 0.1607, loss: 0.9016, grad_norm: 9.7585
2022-04-03 04:29:04,547 - mmdet - INFO - Epoch [1][40/460]	lr: 6.242e-05, eta: 2:09:18, time: 1.357, data_time: 0.020, memory: 31139, loss_rpn_cls: 0.1830, loss_rpn_bbox: 0.0519, loss_cls: 0.2788, acc: 95.6299, l

[>>>>>>>>>>>>>>>>>>>>>>>>>>>] 1206/1206, 13.8 task/s, elapsed: 87s, ETA:     0s

2022-04-03 04:39:57,045 - mmdet - INFO - Evaluating bbox...


...

Loading and preparing results...
DONE (t=0.03s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=4.24s).
Accumulating evaluation results...


2022-04-03 06:51:30,673 - mmdet - INFO - 
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.289
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.450
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.312
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.011
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.067
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.348
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.463
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.463
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.463
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.034
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.192
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=1000

DONE (t=0.95s).


2022-04-03 06:51:32,212 - mmdet - INFO - Now best checkpoint is saved as best_bbox_mAP_epoch_12.pth.
2022-04-03 06:51:32,214 - mmdet - INFO - Best bbox_mAP is 0.2890 at 12 epoch.
2022-04-03 06:51:32,218 - mmdet - INFO - Epoch(val) [12][1206]	bbox_mAP: 0.2890, bbox_mAP_50: 0.4500, bbox_mAP_75: 0.3120, bbox_mAP_s: 0.0110, bbox_mAP_m: 0.0670, bbox_mAP_l: 0.3480, bbox_mAP_copypaste: 0.289 0.450 0.312 0.011 0.067 0.348


learning_rate,███████████████████████████▂▂▂▂▂▂▂▂▂▂▁▁▁
momentum,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/acc,▆▆▃▃▂▄▁▄▃▂▁▆▁▃▁▂▄▃▃▄▄▃▅▅▅▅▅▇▆▇▆▇▇▇▇▇▇█▇█
train/grad_norm,▄▄▅▃▄▃▇▂▃▅▄▃▄▃█▄▃▄▄▃▃▅▂▃▃▃▃▁▂▁▂▂▂▂▂▁▂▁▂▂
train/loss,▇▆▇▆▇▆█▅▆▇▆▃▇▅▆▆▄▅▅▄▄▄▃▃▄▃▄▂▃▂▃▂▂▂▂▁▂▁▂▁
train/loss_bbox,▅▅▆▆█▅█▅▆█▇▂▇▄▇▇▅▅▆▄▅▅▃▃▄▄▃▂▄▁▄▂▂▂▃▁▂▂▃▁
train/loss_cls,▇▆▇▇▇▆█▅▆▇▇▃█▆▇▆▅▅▅▅▄▅▃▃▄▃▄▂▂▂▂▂▂▁▂▁▂▁▂▁
train/loss_rpn_bbox,▅▆▇▅▅▄█▄▅▇▃▄▆▄▄▄▃▄▄▂▃▄▂▂▃▃▃▁▂▂▂▁▂▂▂▁▂▂▂▁
train/loss_rpn_cls,█▆▅▄▄▅▆▃▃▅▃▃▄▂▃▃▃▃▂▂▂▂▂▁▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁
val/bbox_mAP,▁▂▄▄▅▆▆▆████
val/bbox_mAP_50,▁▃▅▅▅▆▆▇████
